# Cell Cycle

Kowalczyk et al. 2015
Cell cycle genes based on Kowalczyk et al. 2015, https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4665007/. Cell cycle is a main source of transcriptional variation between HSCs and cycling cells are more abundand in MPPs compared to LT- and ST-HSCs. 2708 genes, ~32% were significant cell cycle-dependent expression. Of those 92% were up-regulated in cycling versus noncycling cells. 8% were upregulated in noncycling versus cycling cells included some important genes for HSC quiescence which we should not filter out (Stat1, Stat3, Meis1, Pbx1, Klf6, Nfia). G1/S, S, G2, and G2/M phases defined based on functional annotations (The Reference Genome Group of the Gene Ontology Consortium 2009). G1 length varies widely among different cell types, is short specifically in self-renewing cells, and increases with differentiation. 

In [ ]:
library_load <- suppressMessages(
    list(
        # Seurat
        library(Seurat), 

        # Data 
        library(dplyr), 
        library(reshape2), 

        # Plotting
        library(ggplot2),
        library(ggplotify), 
        library(ComplexHeatmap), 
        library(gridExtra),  
        library(RColorBrewer), 
        library(circlize), 
        library(viridis)
    )
)

In [ ]:
options(warn=-1)
ht_opt$message=FALSE

In [ ]:
# Set working directory to project root
setwd("/research/peer/fdeckert/FD20200109SPLENO")

In [ ]:
# Source files
source("plotting_global.R")
source("bin/SeuratQC.R")
source("bin/CellCycleBase.R")
source("bin/CellCycle.R")

# Parameter settings

In [ ]:
# Seurat object 
so_file <- "data/object/qc/so_qc_treatment_reg.rds"

# result dir 
result_dir <- "result/plot/cell_cycle/whitfield/"

# Plotting Theme
ggplot2::theme_set(theme_global_set()) # From project global source()

# Import object

In [ ]:
so <- readRDS(so_file)
so <- merge(so[[1]], so[[2]])
so <- SplitObject(so, split.by = "tissue")

# Import cell cycle genes

In [ ]:
# Import cell cycle genes from Whitefield et al. 2002
cc_genes <- cc_genes_whitfield_load()
# cc_genes <- cc_genes_seurat_load()

# Create CellCycle class object

In [ ]:
cc <- lapply(so, function(so) CellCycle(sample_name=so$tissue[1], so=so, genes=cc_genes))

# Compute correlatin matrix and heat maps 

In [ ]:
cc <- lapply(cc, CorrMatrix)

In [ ]:
source("bin/CellCycle.R")
corr_hm_rho <- lapply(cc, CorrHeatMap, value_var="rho")
corr_hm_fdr <- lapply(cc, CorrHeatMap, value_var="fdr")

In [ ]:
options(repr.plot.width=15, repr.plot.height=10)
corr_hm_rho_grid <- gridExtra::arrangeGrob(grobs=corr_hm_rho, top=textGrob("Correlation matrix (rho)", gp=gpar(fontsize=16, fontface="bold"), y=unit(-1, "npc")), ncol=2) 
ggsave(corr_hm_rho_grid, filename=paste0(result_dir, "corr_hm_rho_grid.png"), width=15, height=10)
grid::grid.draw(corr_hm_rho_grid)

In [ ]:
options(repr.plot.width=15, repr.plot.height=10)
corr_hm_fdr_grid <- gridExtra::arrangeGrob(grobs=corr_hm_fdr, top=textGrob("Correlation matrix (fdr)", gp=gpar(fontsize=16, fontface="bold"), y=unit(-1, "npc")), ncol=2) 
ggsave(corr_hm_fdr_grid, filename=paste0(result_dir, "corr_hm_fdr_grid_o.png"), width=15, height=10)
grid::grid.draw(corr_hm_fdr_grid)

# Weighted gene correlation network analysis

In [ ]:
cc <- lapply(cc, CorrGraph)
cc <- lapply(cc, CorrGraphPrune, fdr_max=0.01, rho_min=0, rho_direction=NULL, degree_min=1)

In [ ]:
graph_grob <- lapply(cc, CorrGraphGrob)

In [ ]:
options(repr.plot.width=10, repr.plot.height=5)
graph_grob_grid <- gridExtra::arrangeGrob(grobs=graph_grob, ncol=2) 
ggsave(graph_grob_grid, filename=paste0(result_dir, "graph_grob_grid.png"), width=10, height=5)
grid::grid.draw(graph_grob_grid)

# Cell cycle phase scattern 

In [ ]:
source("bin/CellCycle.R")
cc <- lapply(cc, ccScore)
cc_score_scatter <- lapply(cc, ccScoreScatter)

In [ ]:
options(repr.plot.width=10, repr.plot.height=5)
cc_score_scatter_grid <- gridExtra::arrangeGrob(grobs=cc_score_scatter, top=textGrob("Cell cycle score scatter", gp=gpar(fontsize=16, fontface="bold"), y=unit(-1, "npc")), ncol=2) 
ggsave(cc_score_scatter_grid, filename=paste0(result_dir, "cc_score_scatter.png"), width=10, height=5)
grid::grid.draw(cc_score_scatter_grid)

# Cell cycle phase heat map 

In [ ]:
cc_score_hm <- lapply(cc, ccScoreHeatMap)

In [ ]:
options(repr.plot.width=20, repr.plot.height=5)
cc_score_hm_grid <- gridExtra::arrangeGrob(grobs=cc_score_hm, top=textGrob("Cell cycle score heat map", gp=gpar(fontsize=16, fontface="bold"), y=unit(-1, "npc")), ncol=1) 
ggsave(cc_score_hm_grid, filename=paste0(result_dir, "cc_score_hm_grid.png"), width=20, height=5)
grid::grid.draw(cc_score_hm_grid)

# Prune cell cycle genes by graph 

In [ ]:
cc_prune <- lapply(cc, CorrGraphModulePrune, fdr_max=0.01, rho_min=0.1, rho_direction=1, degree_min=1)

# Pruned plots

In [ ]:
corr_prune_hm_rho <- lapply(cc_prune, CorrHeatMap, value_var="rho")
corr_prune_hm_fdr <- lapply(cc_prune, CorrHeatMap, value_var="fdr")

In [ ]:
options(repr.plot.width=10, repr.plot.height=10)
corr_prune_hm_rho_grid <- gridExtra::arrangeGrob(grobs=corr_prune_hm_rho, top=textGrob("Correlation matrix prune (rho)", gp=gpar(fontsize=16, fontface="bold"), y=unit(-1, "npc")), ncol=2) 
ggsave(corr_prune_hm_rho_grid, filename=paste0(result_dir, "corr_prune_hm_rho_grid.png"), width=10, height=10)
grid::grid.draw(corr_prune_hm_rho_grid)

In [ ]:
options(repr.plot.width=10, repr.plot.height=10)
corr_prune_hm_fdr_grid <- gridExtra::arrangeGrob(grobs=corr_prune_hm_fdr, top=textGrob("Correlation matrix prune (fdr)", gp=gpar(fontsize=16, fontface="bold"), y=unit(-1, "npc")), ncol=2) 
ggsave(corr_prune_hm_fdr_grid, filename=paste0(result_dir, "corr_prune_hm_fdr_grid.png"), width=10, height=10)
grid::grid.draw(corr_prune_hm_fdr_grid)

In [ ]:
graph_prune_grob <- lapply(cc_prune, CorrGraphGrob)

In [ ]:
options(repr.plot.width=10, repr.plot.height=5)
graph_prune_grob_grid <- gridExtra::arrangeGrob(grobs=graph_prune_grob, ncol=2) 
ggsave(graph_prune_grob_grid, filename=paste0(result_dir, "graph_prune_grob_grid.png"), width=10, height=5)
grid::grid.draw(graph_prune_grob_grid)

In [ ]:
cc_prune <- lapply(cc_prune, ccScore)
cc_prune_score_scatter <- lapply(cc_prune, ccScoreScatter)

In [ ]:
options(repr.plot.width=10, repr.plot.height=5)
cc_prune_score_scatter_grid <- gridExtra::arrangeGrob(grobs=cc_prune_score_scatter, top=textGrob("Cell cycle score scatter prune", gp=gpar(fontsize=16, fontface="bold"), y=unit(-1, "npc")), ncol=2) 
ggsave(cc_prune_score_scatter_grid, filename=paste0(result_dir, "cc_prune_score_scatter_grid.png"), width=10, height=5)
grid::grid.draw(cc_prune_score_scatter_grid)

In [ ]:
cc_prune_score_hm <- lapply(cc_prune, ccScoreHeatMap)

In [ ]:
options(repr.plot.width=20, repr.plot.height=5)
cc_prune_score_hm_grid <- gridExtra::arrangeGrob(grobs=cc_prune_score_hm, top=textGrob("Cell cycle score heat map prune", gp=gpar(fontsize=16, fontface="bold"), y=unit(-1, "npc")), ncol=1) 
ggsave(cc_prune_score_hm_grid, filename=paste0(result_dir, "cc_prune_score_hm_grid.png"), width=20, height=5)
grid::grid.draw(cc_prune_score_hm_grid)

# Plots for lab meeting

In [ ]:
library(patchwork)

In [ ]:
dplot <- function(so) {
    
    dplot_1 <- DimPlot(so, reduction = "umap", group.by = "SCT_snn_res.0.8", label = TRUE) & 
        theme(aspect.ratio = 1, legend.position = "none")
    
    dplot_2 <- DimPlot(so, reduction = "umap", group.by = "fine_labels", label = FALSE) & 
        theme(aspect.ratio = 1, legend.position = "none") & 
        scale_color_manual(values = color$fine_labels, na.value = "dark gray") & 
        guides(color = guide_legend(ncol = 3, override.aes = list(size = 2)))
    
    dplot_3 <- DimPlot(so, reduction = "umap", group.by = "cc_phase_class", label = FALSE) & 
        theme(aspect.ratio = 1, legend.position = "bottom") & 
        scale_color_manual(values = color$cc_phase_class, na.value = "dark gray") & 
        guides(color = guide_legend(ncol = 3, override.aes = list(size = 2))) & 
        ggtitle("Whitfield et al.")

    dplot <- dplot_1 + dplot_2 + dplot_3 + plot_layout(ncol = 3)
    
    return(dplot)
    
}

### Seurat 

In [ ]:
so_new <- readRDS(so_file)

In [ ]:
options(repr.plot.width=10, repr.plot.height=5)
dplot_nacl <- dplot(so_new[["NaCl"]])
dplot_nacl
ggsave(dplot_nacl, filename="result/plot/cc_nacl_seurat.png", width=10, height=5)

In [ ]:
options(repr.plot.width=10, repr.plot.height=5)
dplot_cpg <- dplot(so_new[["CpG"]])
dplot_cpg
ggsave(dplot_cpg, filename="result/plot/cc_cpg_seurat.png", width=10, height=5)

## New

In [ ]:
meta <- merge(cc_prune[[1]]@so, cc_prune[[2]]@so)@meta.data[, "cc_phase_class", drop=FALSE]

In [ ]:
test <- lapply(so_new, function(so) {
    
    so$cc_phase_class <- NULL
    so <- AddMetaData(so, meta)
    return(so)
    
})

In [ ]:
options(repr.plot.width=10, repr.plot.height=5)
dplot_nacl <- dplot(test[["NaCl"]])
dplot_nacl
ggsave(dplot_nacl, filename="result/plot/cc_nacl_whitfield.png", width=10, height=5)

In [ ]:
options(repr.plot.width=10, repr.plot.height=5)
dplot_cpg <- dplot(test[["CpG"]])
dplot_cpg
ggsave(dplot_cpg, filename="result/plot/cc_cpg_whitfield.png", width=10, height=5)